## Random forest classification

## Importing the Libraries

In [4]:
import pandas as pd
import numpy as np 
import seaborn as sns

## Loading the Data Set

In [5]:
df = sns.load_dataset('penguins')
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [6]:
df.shape


(344, 7)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


## Checking Null Values

In [8]:
df.isnull().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

## Removing NUll Value

In [9]:
df.dropna(inplace=True)


## Checking the data set

In [10]:
df.isnull().sum()

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

## Feature Engineering

## One Hot Encoding Transformation categorical data into numeric

In [11]:
df.sex.unique()

array(['Male', 'Female'], dtype=object)

In [12]:
pd.get_dummies(df['sex']).head()

,Female,Male
0,0,1
1,1,0
2,1,0
4,1,0
5,0,1


In [13]:
sex = pd.get_dummies(df['sex'], drop_first=True)
sex.head()

,Male
0,1
1,0
2,0
4,0
5,1


## Applying one-hot encoding to Island

In [14]:
 df.island.unique()

array(['Torgersen', 'Biscoe', 'Dream'], dtype=object)

In [15]:
pd.get_dummies(df['island']).head()

,Biscoe,Dream,Torgersen
0,0,0,1
1,0,0,1
2,0,0,1
4,0,0,1
5,0,0,1


In [16]:
island = pd.get_dummies(df['island'], drop_first = True)
island.head(5)

,Dream,Torgersen
0,0,1
1,0,1
2,0,1
4,0,1
5,0,1


## Concatenate the above data frames to the original df

In [17]:
new_data = pd.concat([df, island, sex], axis=1)

In [18]:
new_data.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,Dream,Torgersen,Male
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,0,1,1
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,0,1,0
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,0,1,0
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female,0,1,0
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male,0,1,1


## Drop the Repeated Columns

In [19]:
new_data.drop(['sex', 'island'], axis=1, inplace=True);


In [20]:
new_data.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,Dream,Torgersen,Male
0,Adelie,39.1,18.7,181.0,3750.0,0,1,1
1,Adelie,39.5,17.4,186.0,3800.0,0,1,0
2,Adelie,40.3,18.0,195.0,3250.0,0,1,0
4,Adelie,36.7,19.3,193.0,3450.0,0,1,0
5,Adelie,39.3,20.6,190.0,3650.0,0,1,1


## Creating Seperate target variable

In [21]:
Y = new_data.species
Y.head()

0    Adelie
1    Adelie
2    Adelie
4    Adelie
5    Adelie
Name: species, dtype: object

In [22]:
Y.unique()

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

## Using the map function to convert categorical values into numeric

In [23]:
Y=Y.map({'Adelie':0,'Chinstrap':1,'Gentoo':2})
Y.head()

0    0
1    0
2    0
4    0
5    0
Name: species, dtype: int64

## Dropping the Target Variable : Species

In [24]:
new_data.drop('species', inplace = True, axis=1)

In [25]:
new_data.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,Dream,Torgersen,Male
0,39.1,18.7,181.0,3750.0,0,1,1
1,39.5,17.4,186.0,3800.0,0,1,0
2,40.3,18.0,195.0,3250.0,0,1,0
4,36.7,19.3,193.0,3450.0,0,1,0
5,39.3,20.6,190.0,3650.0,0,1,1


In [26]:
X = new_data

## Splitting the Data into Training and Test Data

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=0)

In [30]:
print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('Y_train', Y_train.shape)
print('Y_test', Y_test.shape)

X_train (233, 7)
X_test (100, 7)
Y_train (233,)
Y_test (100,)


## Training Random Forest Classification on Training Set

In [32]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=5, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=5, random_state=0)

In [33]:
Y_pred = classifier.predict(X_test)
Y_pred

array([0, 0, 2, 0, 0, 0, 1, 2, 2, 1, 2, 0, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0,
       2, 2, 2, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 1, 0, 2, 2, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 2, 0, 0,
       2, 2, 1, 2, 2, 1, 2, 1, 0, 2, 0, 2, 0, 2, 1, 2, 2, 2, 1, 2, 1, 0,
       0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2], dtype=int64)

## Confusion matrix

In [34]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

In [35]:
cm = confusion_matrix(Y_test, Y_pred)
print(cm)

[[48  0  0]
 [ 2 14  0]
 [ 0  0 36]]


In [36]:
accuracy_score(Y_test, Y_pred)

0.98

In [37]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        48
           1       1.00      0.88      0.93        16
           2       1.00      1.00      1.00        36

    accuracy                           0.98       100
   macro avg       0.99      0.96      0.97       100
weighted avg       0.98      0.98      0.98       100



## Try with Different number of trees and gini criteria

In [39]:
from sklearn.ensemble import RandomForestClassifier
classifier =  RandomForestClassifier(n_estimators=12, criterion = 'gini', random_state = 0)
classifier.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=12, random_state=0)

In [40]:
Y_pred = classifier.predict(X_test)

In [41]:
accuracy_score(Y_test, Y_pred)

0.98